In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
#gpt4omini, llama317B
generation_model = 'llama317B'

In [3]:
import os
from openai import OpenAI
from pydantic import BaseModel
import json

In [4]:
with open('../Datasets/all_prompts_task1.json', 'r') as file:
    user_prompt = file.read()

In [5]:
if(generation_model == 'gpt4omini'):
    client = OpenAI(
        api_key = os.getenv('OPEN_AI_API')
    )

    def create_travel_plan(system_prompt, user_prompt):
        chat_completion = client.chat.completions.create(
            model="gpt-4o-mini",
            temperature=0,
            messages=[
                {"role": "system","content": system_prompt,},
                {"role": "user","content": user_prompt,}
            ]
        )

        return chat_completion.choices[0].message.content


    for filename in os.listdir('Prompts/prompts_routesOP'):
        index = filename[:-4][7:]
        with open(f'Prompts/prompts_routesOP/{filename}', 'r') as file:
            system_prompt = file.read()
            plan = create_travel_plan(system_prompt, user_prompt)
            with open(f'Outputs/plans/gpt4omini/Plan_{index}.txt', 'w') as f:
                f.write(plan)

In [6]:
from huggingface_hub import login
login()

In [7]:
torch.cuda.get_device_name(device=None)

'NVIDIA A100-SXM4-80GB'

In [8]:
if(generation_model == 'llama317B'):
    import transformers
    import torch
    
    model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
    pipeline = transformers.pipeline(
        "text-generation",
        model=model_id,
        model_kwargs={"torch_dtype": torch.bfloat16},
        device="cuda",
    )
    
    for filename in os.listdir('Prompts/prompts_routesOP'):
        
        #testing purpose
        if(filename == 'Prompt_3.txt' or filename == 'Prompt_9.txt' or filename == 'Prompt_99.txt'):
            continue

        index = filename[:-4][7:]
        with open(f'Prompts/prompts_routesOP/{filename}', 'r') as file:
            system_prompt = file.read()
        
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]

        outputs = pipeline(
            messages,
            max_new_tokens=5000, # the llama 3.1 won't increase the detail with more token limit, neither decrese detail with less token, so just create a token limit that won't cut off the text.
            do_sample = False
        )

        with open(f'Outputs/plans/Llama317B/Plan_{index}.txt', 'w') as f:
                    f.write(outputs[0]["generated_text"][-1]['content'])

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda
/home/jqf8qm/miniforge3/envs/torchgpu/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/jqf8qm/miniforge3/envs/torchgpu/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.46 GiB. GPU 0 has a total capacity of 79.14 GiB of which 217.62 MiB is free. Process 299107 has 40.71 GiB memory in use. Including non-PyTorch memory, this process has 38.09 GiB memory in use. Of the allocated memory 32.19 GiB is allocated by PyTorch, and 5.40 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)